In [1]:
import pandas as pd

In [2]:
import plotly.express as px
from dash.dependencies import Input, Output, State

In [3]:
df = pd.read_csv('data/pos_data.csv', header=[0,1,2,3,4,5], parse_dates=[0])

In [4]:
df.head()

Unnamed: 0_level_0         週次                                         \
  Unnamed: 0_level_1      00_全国                                          
  Unnamed: 0_level_2 05_家電大型専門店                                          
  Unnamed: 0_level_3    00_大分類計          70_家電                           
  Unnamed: 0_level_4    00_中分類計        00_中分類計        71_カラーテレビ          
  Unnamed: 0_level_5     販売金額指数 前年比（％）  販売金額指数 前年比（％）    販売金額指数 前年比（％）   
0         2020-06-28     183.98  22.16  183.98  22.16    171.05  42.61   
1         2020-06-21     179.44  40.77  179.44  40.77    160.56  39.47   
2         2020-06-14     203.27  70.97  203.27  70.97    152.13  37.58   
3         2020-06-07     181.18  43.32  181.18  43.32    138.10  40.03   
4         2020-05-31     145.50   6.55  145.50   6.55    109.33  25.49   

                          ...                                               \
                          ... 15_中国・四国・九州・沖縄                                 
                          ...     05_家電大型専門店                                 
                          ...          70_家電                                 
  72_パソコン         73_冷蔵庫  ...      71_カラーテレビ        72_パソコン         73_冷蔵庫   
   販売金額指数 前年比（％）  販売金額指数  ...         販売金額指数 前年比（％）  販売金額指数 前年比（％）  販売金額指数   
0  107.42  21.85  192.71  ...         175.11  38.42  115.33  20.27  188.50   
1   99.07  24.95  178.18  ...         165.09  32.76  105.78  22.44  182.56   
2   95.02  24.55  180.42  ...         161.07  40.95  102.73  30.69  180.33   
3   94.87  37.17  160.64  ...         155.91  58.54  100.51  55.21  159.34   
4   90.11  28.75  130.51  ...         107.17  17.91   92.77  39.42  129.41   

                                         
                                         
                                         
                                         
          74_洗濯機        75_エアコン          
  前年比（％）  販売金額指数 前年比（％）  販売金額指数  前年比（％）  
0  23.38  191.43  32.50  259.06    5.12  
1  33.52  182.38  37.95  267.88   53.83  
2  44.10  181.82  50.11  360.16  121.92  
3  30.00  162.70  42.33  268.70   31.57  
4  13.32  144.20  26.55  204.35   11.00  

[5 rows x 747 columns]

In [5]:
df.shape

(443, 747)

In [6]:
melted_df = df.melt(id_vars=[df.columns[0]])

cols_name = ['date', 'data_time', 'area', 'store', 'variable_3', 'variable_4', 'index_or_yoy', 'value']
melted_df.columns = cols_name

In [7]:
for col in melted_df.columns[1:-1]:
    print(f'{col}: {melted_df[col].unique()}')

data_time: ['週次']
area: ['00_全国' '01_北海道' '02_東北' '03_関東' '04_中部' '05_近畿' '06_中国' '07_四国' '08_九州'
 '09_沖縄' '11_北海道・東北' '12_関東・甲越' '13_東海・北陸' '14_近畿' '15_中国・四国・九州・沖縄']
store: ['05_家電大型専門店' '01_スーパーマーケット' '02_コンビニエンスストア' '03_ホームセンター' '04_ドラッグストア']
variable_3: ['00_大分類計' '70_家電' '10_食品' '20_飲料' '30_雑貨' '40_化粧品' '50_ヘルスケア' '60_その他']
variable_4: ['00_中分類計' '71_カラーテレビ' '72_パソコン' '73_冷蔵庫' '74_洗濯機' '75_エアコン' '11_主食'
 '12_調味料' '13_加工食品' '14_菓子' '15_嗜好品' '21_嗜好飲料' '22_清涼飲料' '23_アルコール飲料'
 '31_オーラルケア' '32_パーソナルケア' '33_ハウスホールド' '34_紙製品' '35_その他雑貨' '36_ペット'
 '41_基礎化粧品' '42_メイクアップ化粧品' '43_その他化粧品' '51_医薬品' '52_健康関連品' '53_ベビー'
 '61_たばこ']
index_or_yoy: ['販売金額指数' '前年比（％）']


In [8]:
from jupyter_dash import JupyterDash
import dash_html_components as html 
import dash_core_components as dcc 
from dash.dependencies import Input, Output

In [9]:
app = JupyterDash(__name__)

app.layout = html.Div([
    
    dcc.RadioItems(
        id='my_radio',
        options=[{'value': i, 'label': i} for i in melted_df['index_or_yoy'].unique()],
        value='販売金額指数'
    ),
    
    dcc.Dropdown(
        id='area_dropdown',
        options=[{'value': area, 'label': area} for area in melted_df['area'].unique()],
        value=['00_全国'],
        multi=True
    ),
    
    dcc.Dropdown(id='my_dropdown',
                options=[{'label': v, 'value': v} for v in melted_df['variable_4'].unique()],
                 value = '71_カラーテレビ'
                ),
    
    dcc.Graph(id='my_graph')
    
])

@app.callback(Output('my_graph', 'figure'), 
             [Input('my_radio', 'value'), Input('area_dropdown', 'value'), Input('my_dropdown', 'value')]
             )
def update_graph(radio_value, area_value, drop_value):
    selected_df = melted_df[melted_df['index_or_yoy'] == radio_value]
    selected_df = selected_df[selected_df['area'].isin(area_value)]
    selected_df = selected_df[selected_df['variable_4'] == drop_value]
    return px.line(selected_df, x='date', y='value', color='area')

app.run_server(mode='inline')

In [10]:
pd.set_option('max_columns', 1000)
pd.set_option('max_rows', 200)

In [11]:
df.head()

Unnamed: 0_level_0         週次                                         \
  Unnamed: 0_level_1      00_全国                                          
  Unnamed: 0_level_2 05_家電大型専門店                                          
  Unnamed: 0_level_3    00_大分類計          70_家電                           
  Unnamed: 0_level_4    00_中分類計        00_中分類計        71_カラーテレビ          
  Unnamed: 0_level_5     販売金額指数 前年比（％）  販売金額指数 前年比（％）    販売金額指数 前年比（％）   
0         2020-06-28     183.98  22.16  183.98  22.16    171.05  42.61   
1         2020-06-21     179.44  40.77  179.44  40.77    160.56  39.47   
2         2020-06-14     203.27  70.97  203.27  70.97    152.13  37.58   
3         2020-06-07     181.18  43.32  181.18  43.32    138.10  40.03   
4         2020-05-31     145.50   6.55  145.50   6.55    109.33  25.49   

                                                                             \
                                                                              
                                                               01_スーパーマーケット   
                                                                    00_大分類計   
  72_パソコン         73_冷蔵庫         74_洗濯機        75_エアコン              00_中分類計   
   販売金額指数 前年比（％）  販売金額指数 前年比（％）  販売金額指数 前年比（％）  販売金額指数  前年比（％）       販売金額指数   
0  107.42  21.85  192.71  30.19  193.22  32.83  247.39    5.23       108.03   
1   99.07  24.95  178.18  35.51  179.26  37.66  265.96   52.83       112.87   
2   95.02  24.55  180.42  46.86  178.73  42.14  374.83  142.41       107.06   
3   94.87  37.17  160.64  28.94  162.54  31.06  321.16   59.32       110.17   
4   90.11  28.75  130.51   7.51  140.42  16.47  238.77   -7.33       109.64   

                                                                              \
                                                                               
                                                                               
           10_食品                                                               
         00_中分類計          11_主食         12_調味料        13_加工食品          14_菓子   
  前年比（％）  販売金額指数 前年比（％）  販売金額指数 前年比（％）  販売金額指数 前年比（％）  販売金額指数 前年比（％）  販売金額指数   
0   3.63  110.82   4.59  110.48   2.71  115.92   6.82  110.19   5.45   91.94   
1   8.86  114.16   7.94  111.69   5.38  122.25  12.52  113.46   8.44   95.74   
2   1.13  110.28   2.25  108.66   1.24  116.69   3.55  109.49   2.30   91.28   
3   6.97  113.31   7.91  111.91   6.74  120.39  11.88  112.11   7.67   94.20   
4   4.39  114.65   7.58  113.78   5.06  120.13   9.87  113.77   8.83  101.71   

                                                                      \
                                                                       
                                                                       
                          20_飲料                                        
          15_嗜好品        00_中分類計        21_嗜好飲料        22_清涼飲料          
  前年比（％）  販売金額指数 前年比（％）  販売金額指数 前年比（％）  販売金額指数 前年比（％）  販売金額指数 前年比（％）   
0   0.44  139.35   6.70  107.64   3.04   91.49   7.33  129.46   2.60   
1   3.32  141.52   8.86  115.93  12.32   96.88  15.68  132.73   6.71   
2  -6.49  140.95  15.95  108.64   2.01   86.77  -1.34  129.43  10.26   
3   2.00  144.09  11.62  112.49   8.30   98.52  19.47  133.54   7.55   
4   8.06  133.64   2.96  108.78   2.69   99.44   8.36  122.75  -3.73   

                                                                       \
                                                                        
                                                                        
                      30_雑貨                                             
  23_アルコール飲料        00_中分類計        31_オーラルケア        32_パーソナルケア          
      販売金額指数 前年比（％）  販売金額指数 前年比（％）    販売金額指数 前年比（％）     販売金額指数 前年比（％）   
0      95.14   2.80  100.58   2.90    103.23  -2.52      93.88   2.40   
1     107.41  17.11  104.40   7.59    103.76  -0.98      99.48   9.11   
2      97.81  -4.12   95.41  -3.02     95.47 -10.94      91

In [13]:
df.columns[0]

('Unnamed: 0_level_0',
 'Unnamed: 0_level_1',
 'Unnamed: 0_level_2',
 'Unnamed: 0_level_3',
 'Unnamed: 0_level_4',
 'Unnamed: 0_level_5')

In [18]:
df_melt = df.melt(id_vars=[df.columns[0]])
df_melt.columns = ['date', '週次', '地域', '部門', '大分類', '中分類', '指数or比率', 'value']

In [22]:
df_melt_index = df_melt[df_melt['指数or比率'] == '販売金額指数']
df_melt_index

,date,週次,地域,部門,大分類,中分類,指数or比率,value
0,2020-06-28,週次,00_全国,05_家電大型専門店,00_大分類計,00_中分類計,販売金額指数,183.98
1,2020-06-21,週次,00_全国,05_家電大型専門店,00_大分類計,00_中分類計,販売金額指数,179.44
2,2020-06-14,週次,00_全国,05_家電大型専門店,00_大分類計,00_中分類計,販売金額指数,203.27
3,2020-06-07,週次,00_全国,05_家電大型専門店,00_大分類計,00_中分類計,販売金額指数,181.18
4,2020-05-31,週次,00_全国,05_家電大型専門店,00_大分類計,00_中分類計,販売金額指数,145.5
...,...,...,...,...,...,...,...,...
330030,2012-02-05,週次,15_中国・四国・九州・沖縄,05_家電大型専門店,70_家電,75_エアコン,販売金額指数,NaN
330031,2012-01-29,週次,15_中国・四国・九州・沖縄,05_家電大型専門店,70_家電,75_エアコン,販売金額指数,NaN
330032,2012-01-22,週次,15_中国・四国・九州・沖縄,05_家電大型専門店,70_家電,75_エアコン,販売金額指数,NaN
330033,2012-01-15,週次,15_中国・四国・九州・沖縄,05_家電大型専門店,70_家電,75_エアコン,販売金額指数,NaN


In [31]:
df_melt_jp = df_melt_index[df_melt_index['地域'] == '00_全国']
df_melt_jp_recent = df_melt_jp[df_melt_jp['date'] == '2020-06-28']
df_melt_jp_recent = df_melt_jp_recent.reset_index(drop=True)
df_melt_jp_recent['部門'].unique()

array(['05_家電大型専門店', '01_スーパーマーケット', '02_コンビニエンスストア', '03_ホームセンター',
       '04_ドラッグストア'], dtype=object)

In [33]:
df_melt_jp_recent

,date,週次,地域,部門,大分類,中分類,指数or比率,value
0,2020-06-28,週次,00_全国,05_家電大型専門店,00_大分類計,00_中分類計,販売金額指数,183.98
1,2020-06-28,週次,00_全国,05_家電大型専門店,70_家電,00_中分類計,販売金額指数,183.98
2,2020-06-28,週次,00_全国,05_家電大型専門店,70_家電,71_カラーテレビ,販売金額指数,171.05
3,2020-06-28,週次,00_全国,05_家電大型専門店,70_家電,72_パソコン,販売金額指数,107.42
4,2020-06-28,週次,00_全国,05_家電大型専門店,70_家電,73_冷蔵庫,販売金額指数,192.71
...,...,...,...,...,...,...,...,...
110,2020-06-28,週次,00_全国,04_ドラッグストア,40_化粧品,43_その他化粧品,販売金額指数,96.87
111,2020-06-28,週次,00_全国,04_ドラッグストア,50_ヘルスケア,00_中分類計,販売金額指数,85.2
112,2020-06-28,週次,00_全国,04_ドラッグストア,50_ヘルスケア,51_医薬品,販売金額指数,70.43
113,2020-06-28,週次,00_全国,04_ドラッグストア,50_ヘルスケア,52_健康関連品,販売金額指数,139.15
